In [1]:
import numpy as np
import data
from sklearn.decomposition import PCA
from sklearn.neighbors import LocalOutlierFactor as LOF
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from xgboost import XGBClassifier
import nn
from sklearn.model_selection import train_test_split, GridSearchCV
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
train_data = data.Dataset('train.csv')
#train_data.to_one_hot()


In [8]:
pipe = Pipeline([
    ('pca',PCA()),
    ('lr',LogisticRegression())
])

param_grid = {
    'pca__n_components': [64,128,256,512],
    'lr__C': [0.01,0.1,1,10,100,1000],
    'lr__penalty': ['none','l2'],
    'lr__max_iter': [10000]
}

grid = GridSearchCV(pipe,param_grid,cv=5,verbose=1,n_jobs=-1)
grid.fit(train_x,train_y)


Fitting 5 folds for each of 48 candidates, totalling 240 fits


/home/sid/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/home/sid/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/home/sid/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/home/sid/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/home/sid/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/home/sid/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Se

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('pca', PCA()),
                                       ('lr', LogisticRegression())]),
             n_jobs=-1,
             param_grid={'lr__C': [0.01, 0.1, 1, 10, 100, 1000],
                         'lr__max_iter': [10000], 'lr__penalty': ['none', 'l2'],
                         'pca__n_components': [64, 128, 256, 512]},
             verbose=1)

In [9]:
grid.best_params_

{'lr__C': 1,
 'lr__max_iter': 10000,
 'lr__penalty': 'l2',
 'pca__n_components': 512}

In [10]:
grid.score(test_x,test_y)

0.8114754098360656

In [84]:
lof = LOF(n_neighbors=5)
outlier_labels = lof.fit_predict(train_data.x)
train_data.x = train_data.x[outlier_labels == 1]
train_data.y = train_data.y[outlier_labels == 1]

In [54]:
scaler = StandardScaler()
train_data.x = scaler.fit_transform(train_data.x)

In [85]:
pca = PCA(n_components=300)
pca.fit(train_data.x)
train_data.x = pca.transform(train_data.x)
np.sum(pca.explained_variance_ratio_)


0.9869384496315264

In [5]:
train_x,test_x,train_y,test_y = train_test_split(train_data.x,train_data.y,test_size=0.1)

In [57]:
print(train_x.shape,train_y.shape)

(1084, 256) (1084,)


In [86]:
nn.train(train_data.x,train_data.y,validation_split=0.1,epochs=200,weight_decay=1.5*10e-6)

Epoch [1/200], Loss: 2.9765
Epoch [2/200], Loss: 2.9422
Epoch [3/200], Loss: 2.8863
Epoch [4/200], Loss: 2.8386
Epoch [5/200], Loss: 2.8149
Epoch [6/200], Loss: 2.7104
Epoch [7/200], Loss: 2.6909
Epoch [8/200], Loss: 2.5839
Epoch [9/200], Loss: 2.5155
Epoch [10/200], Loss: 2.4122
Epoch [11/200], Loss: 2.3403
Epoch [12/200], Loss: 2.4247
Epoch [13/200], Loss: 2.3476
Epoch [14/200], Loss: 2.4392
Epoch [15/200], Loss: 2.4833
Epoch [16/200], Loss: 2.3603
Epoch [17/200], Loss: 2.2872
Epoch [18/200], Loss: 2.2962
Epoch [19/200], Loss: 2.2487
Epoch [20/200], Loss: 2.2871
Epoch [21/200], Loss: 2.2569
Epoch [22/200], Loss: 2.2725
Epoch [23/200], Loss: 2.2568
Epoch [24/200], Loss: 2.3059
Epoch [25/200], Loss: 2.2744
Epoch [26/200], Loss: 2.1937
Epoch [27/200], Loss: 2.2382
Epoch [28/200], Loss: 2.1617
Epoch [29/200], Loss: 2.1619
Epoch [30/200], Loss: 2.1594
Epoch [31/200], Loss: 2.2384
Epoch [32/200], Loss: 2.1522
Epoch [33/200], Loss: 2.1970
Epoch [34/200], Loss: 2.1765
Epoch [35/200], Loss: 2

tensor(0.8430)

In [81]:


lda = LinearDiscriminantAnalysis()
lda.fit(train_x,train_y)
tran_x = lda.transform(train_x)
trantest_x = lda.transform(test_x)

clf = RFC(min_samples_split=8,bootstrap=False,criterion='entropy',max_depth=20,max_features='sqrt',min_samples_leaf=1,n_estimators=1000)

clf.fit(tran_x,train_y)
print(clf.score(tran_x,train_y))
clf.score(trantest_x,test_y)

1.0


0.71900826446281

In [87]:
import torch
m = nn.Net(train_data.x.shape[1])
m.load_state_dict(torch.load('model.pth'))


<All keys matched successfully>

In [88]:
m(torch.from_numpy(test_x).float()).argmax(1).numpy()

RuntimeError: mat1 and mat2 shapes cannot be multiplied (121x256 and 300x20)

In [71]:
print(clf.best_score_)
print(clf.best_params_)

0.6513184843830006
{'bootstrap': False, 'criterion': 'entropy', 'max_depth': None, 'min_samples_split': 8}


In [12]:
test_data = data.Dataset('test.csv')
pred = grid.predict(test_data.x)
data.write_to_csv('submission.csv',pred,train_data.get_cat_to_label())